In [1]:
# # fetch username foler directory
# import os

# first = len("c:\\users\\")
# print(os.path.abspath(os.getcwd()))
# pwd = os.path.abspath(os.getcwd())
# second = pwd[first:len(pwd)]
# third = second.find("\\")
# pc_username_folder = second[0:third]
# print("username: "+ pc_username_folder)

In [2]:
save_loc =fr'C:\Users\rsimon\Desktop\pharmacist_list_selenium.csv'

***
# 🐍

In [3]:
#importing all libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait,Select
#from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from bs4 import BeautifulSoup as soup
from selenium.webdriver.chrome.options import Options # background run option
import time
from datetime import datetime
from IPython.display import Markdown, display # for printmd
# pip install selenium # for first time run, 
from selenium.webdriver.common.action_chains import ActionChains

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import os

from pathlib import Path

from openpyxl import Workbook
from openpyxl import Workbook,load_workbook

versions of the core libraries used
- selenium version: 3.141.0
-pandas version: 1.0.1
-BeautifulSoup version: 4.8.2
-requests version: 2.22.0
-python version: 3.7.3

In [4]:
# CHROME Driver Setttings
WINDOW_SIZE = "1920,1080"
chromeOptions = webdriver.ChromeOptions()
prefs = {'profile.managed_default_content_settings.images':2}# no image loading settting, for reduced bandwith
chromeOptions.add_experimental_option("prefs", prefs)
#chromeOptions.add_argument("--headless") # invisible browser
chromeOptions.add_argument("--window-size=%s" % WINDOW_SIZE)
# Pass the argument 1 to allow and 2 to block (Pop Up Notifications)
chromeOptions.add_experimental_option("prefs", { "profile.default_content_setting_values.notifications": 2 })
#chromeOptions.add_argument("--proxy-server=socks5://127.0.0.1:1080" )  # Apply proxy settings (if getting error "Your IP cannot Login")
d = webdriver.Chrome(options=chromeOptions)

wait = WebDriverWait(d, 60)

In [5]:
Link = 'https://idbop.mylicense.com/verification/Search.aspx'

In [6]:
# load the page
try:
    d.get(Link)
except:
    pass

In [7]:
first_mid_name=''
last_name='L*'
city=''
state =''
country=''
zip_code=''

license_type = 'Pharmacist'
License_Number=''
License_status=''

In [8]:
def click_searchButton():
    searchButton = d.find_element_by_xpath('//input[@id="sch_button"]')
    searchButton.click()

In [9]:
def lic_type(Ltype):
    selm = d.find_element_by_id('t_web_lookup__license_type_name')
    sel_lc = Select(selm)
    sel_lc.select_by_visible_text(f'{Ltype}')

In [10]:
def lic_status(lstat):
    selm = d.find_element_by_id('t_web_lookup__license_status_name')
    sel_lc = Select(selm)
    sel_lc.select_by_visible_text(f'{lstat}')

In [11]:
def state_address(state):
    selm = d.find_element_by_id('t_web_lookup__addr_state')
    sel_lc = Select(selm)
    sel_lc.select_by_visible_text(f'{state}')

In [12]:
if license_type  !='':
    print("encoding license Type")
    lic_type(license_type)

encoding license Type


In [13]:
if first_mid_name  !='':
    print("encoding first/mid name")
    fm_name = d.find_element_by_id("t_web_lookup__first_name")
    fm_name.clear()
    time.sleep(1)
    fm_name.send_keys(firs_mid_name)   

In [14]:
if last_name  !='':
    print("encoding Last name")
    fm_name = d.find_element_by_id("t_web_lookup__last_name")
    fm_name.clear()
    time.sleep(1)
    fm_name.send_keys(last_name)  

encoding Last name


In [15]:
if city  !='':
    print("encoding city")
    fm_name = d.find_element_by_id("t_web_lookup__addr_city")
    fm_name.clear()
    time.sleep(1)
    fm_name.send_keys(city)  

In [16]:
if zip_code  !='':
    print("encoding city")
    fm_name = d.find_element_by_id("t_web_lookup__addr_city")
    fm_name.clear()
    time.sleep(1)
    fm_name.send_keys(zip_code)  

In [17]:
if country  !='':
    print("encoding city")
    fm_name = d.find_element_by_id("t_web_lookup__addr_zipcode")
    fm_name.clear()
    time.sleep(1)
    fm_name.send_keys(city)

In [18]:
click_searchButton()

In [19]:
# format the fetch list into  columns.
def list_format(list_name, step):
    return [list_name[i::step] for i in range(step)]
#print(list_format(data,4))

In [20]:
def extract_table():
    soup1 = soup(d.page_source, 'html.parser') # transfer to beautifull soup
    table = soup1.find('table', attrs={'id':'datagrid_results'})
    tbody=table.find('tbody')
    td_as_base1 = tbody.find_all('td') 
    data2 = []
    for cell in td_as_base1:
            if cell.text=="":
                #print("empty string skipped")
                continue
            #if cell.text.startswith('\n'):
            if '\\' in  r"%r" % cell.text:
                #print("slash skipped")
                continue
            insertdata = cell.text.strip()
            data2.append(insertdata)
    return data2

In [21]:
def count_pages():
    soup1 = soup(d.page_source, 'html.parser')
    paginationtags = soup1.find_all('a') # all trs
    pages = [tag.text.strip() for tag in paginationtags if tag.text.isnumeric()]
    return pages

In [22]:
pages = count_pages()

In [23]:
# extract what is on the first page
pages_list =[] # raw containter for all the pages 
first_page =extract_table()
pages_list.append(first_page)

In [24]:
# loop around the other pages
for page_no in pages:
    print("extracting page "+str(page_no))
    pagination_xpath = fr'//a[text()="{page_no}"]'
    pagination = wait.until(EC.element_to_be_clickable((By.XPATH, pagination_xpath)))
    pagination.click()
    time.sleep(1)
    curr_table = extract_table()
    pages_list.append(curr_table)
    time.sleep(1)

extracting page 2
extracting page 3
extracting page 4
extracting page 5
extracting page 6
extracting page 7
extracting page 8
extracting page 9


In [25]:
# consolitdate al in one list
finalList =[]
for item in pages_list:
    for cells in  item:
        finalList.append(cells)

In [26]:
formattedFinalList = list_format(finalList,4)

In [27]:
# resconstruct the table
print("constructing table")
Name = formattedFinalList[0]
License_NO = formattedFinalList[1]
Licence_type =   formattedFinalList[2]
Status =  formattedFinalList[3]
phamacist_list_dict = {'Name': Name, 'License_NO': License_NO, 'Licence_type': Licence_type,"Status":Status}
final_list =  pd.DataFrame(phamacist_list_dict)
#len(final_list)

constructing table


In [28]:
print("saving file")
filepath = Path(save_loc)  
filepath.parent.mkdir(parents=True, exist_ok=True)  
final_list.to_csv(filepath)

saving file


PermissionError: [WinError 5] Access is denied: 'C:\\Users\\rsimon'